## BERT新闻分类

使用BERT模型对新闻标题短文本分类

In [1]:
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences  # padding句子用
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


### GPU

检查GPU状态

In [2]:
print("Is CUDA available: ", torch.cuda.is_available())
n_gpu = torch.cuda.device_count()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU numbers: ", n_gpu)
print("device_name: ", torch.cuda.get_device_name(0))

Is CUDA available:  True
GPU numbers:  2
device_name:  Tesla M40 24GB


### 数据处理

#### 读取数据

放在 `./datasets/THUCNews/train.txt`中

In [3]:
file = "./datasets/THUCNews/train.txt"

with open(file, encoding="utf-8") as f:
    sentences_and_labels = [line for line in f.readlines()]
f.close()

In [4]:
# 前几句
sentences_and_labels[0:10]

['中华女子学院：本科层次仅1专业招男生\t3\n',
 '两天价网站背后重重迷雾：做个网站究竟要多少钱\t4\n',
 '东5环海棠公社230-290平2居准现房98折优惠\t1\n',
 '卡佩罗：告诉你德国脚生猛的原因 不希望英德战踢点球\t7\n',
 '82岁老太为学生做饭扫地44年获授港大荣誉院士\t5\n',
 '记者回访地震中可乐男孩：将受邀赴美国参观\t5\n',
 '冯德伦徐若瑄隔空传情 默认其是女友\t9\n',
 '传郭晶晶欲落户香港战伦敦奥运 装修别墅当婚房\t1\n',
 '《赤壁OL》攻城战诸侯战硝烟又起\t8\n',
 '“手机钱包”亮相科博会\t4\n']

数据以 `table` 分割，所以用 `split('\t')`：

In [5]:
seq, label = sentences_and_labels[2].split('\t')
print(seq)
print(label)

东5环海棠公社230-290平2居准现房98折优惠
1



In [6]:
sentences = []
labels = []

for sentence_with_label in sentences_and_labels:
    sentence, label = sentence_with_label.split('\t')
    sentences.append(sentence)
    labels.append(label)

In [7]:
print(sentences[0:5])
print(labels[0:5])

['中华女子学院：本科层次仅1专业招男生', '两天价网站背后重重迷雾：做个网站究竟要多少钱', '东5环海棠公社230-290平2居准现房98折优惠', '卡佩罗：告诉你德国脚生猛的原因 不希望英德战踢点球', '82岁老太为学生做饭扫地44年获授港大荣誉院士']
['3\n', '4\n', '1\n', '7\n', '5\n']


#### 按字拆分：
    
    
    
按照BERT的要求，我们需要使用输入为：
```
[CLS]<句子A>[SEP]<句子B>[SEP]
```
这样的形式，但是很明显我们的新闻标题是不适合这样分开的，所以我们的输入形式是：
```
[CLS]<句子A>[SEP]
```

BERT不需要分词，我们只要直接将他们转换为 `vocab.txt` 字典中对应的字索引即可，包括`[CLS]`和`[SEP]`。

例如：

In [8]:
tokenizer = BertTokenizer.from_pretrained('./bert-chinese/', do_lower_case=True)
tokenizer

In [9]:
tokenized_texts = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

In [10]:
# 这句话的input_ids
print(f"Tokenize 前的第一句话：\n{sentences[0]}\n")
print(f"Tokenize 后的第一句话: \n{tokenized_texts[0]}")

Tokenize 前的第一句话：
中华女子学院：本科层次仅1专业招男生

Tokenize 后的第一句话: 
[101, 704, 1290, 1957, 2094, 2110, 7368, 8038, 3315, 4906, 2231, 3613, 788, 122, 683, 689, 2875, 4511, 4495, 102]


以上面的例子说明：

首先encode包含了两个动作，

**第一**，对句子

    ```中华女子学院：本科层次仅1专业招男生```

的前后添加标签，即：

    ```[CLS]中华女子学院：本科层次仅1专业招男生[SEP]```

**第二**，将添加标签后的句子按字符（标点符号也单独算一个字符）分开，然后转换为 `input_ids`：

    ```[101, 704, 1290, 1957, 2094, 2110, 7368, 8038, 3315, 4906, 2231, 3613, 788, 122, 683, 689, 2875, 4511, 4495, 102]```

> 需要说明的是，其中 `101` 是 `[CLS]` 的索引，`102` 是 `[SEP]` 的索引。

上述过程称为 `tokenized`。

In [11]:
print (len(tokenized_texts))  # 180000句话

180000


#### Padding

为了保证输入长度的统一，我们需要对句子进行padding。

本例中采用的是新闻标题，所以标题不会太长，我们限定为 `32` 个字符。

一旦标题长度超过32个字符，则会截断超过部分不用；如果不足 `32` 个字符，则执行 `pad_sequences` （即`padding`）操作。

In [12]:
# 句子最长长度
MAX_LEN = 32

# 输入padding
# 此函数在keras里面
input_ids = pad_sequences([txt for txt in tokenized_texts],
                          maxlen=MAX_LEN, 
                          dtype="long", 
                          truncating="post", 
                          padding="post")

In [13]:
print(f"Tokenize 前的第一句话：\n\n{sentences[0]}\n\n")
print(f"Tokenize 后的第一句话: \n\n{tokenized_texts[0]}\n\n")
print(f"Padding 后的第一句话： \n\n{input_ids[0]}")

Tokenize 前的第一句话：

中华女子学院：本科层次仅1专业招男生


Tokenize 后的第一句话: 

[101, 704, 1290, 1957, 2094, 2110, 7368, 8038, 3315, 4906, 2231, 3613, 788, 122, 683, 689, 2875, 4511, 4495, 102]


Padding 后的第一句话： 

[ 101  704 1290 1957 2094 2110 7368 8038 3315 4906 2231 3613  788  122
  683  689 2875 4511 4495  102    0    0    0    0    0    0    0    0
    0    0    0    0]


其实 `padding` 之后还可转换回来，

很容易看出每个字，包括`[PAD]`、`[CLS]`、`[SEP]`所在的位置:

In [14]:
# 转换回来
raw_texts = [tokenizer.decode(input_ids[0])]
print(raw_texts)
print(len(raw_texts))

['[CLS] 中 华 女 子 学 院 ： 本 科 层 次 仅 1 专 业 招 男 生 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']
1


### BERT的输入准备

#### 注意力mask（attention masks）：

BERT 模型的核心是 Transformer 结构，其中很重要的一点就是 self-attention 结构。

BERT-Chinese 模型同 BERT-base 模型结构一致，每层有12个自注意头，为了不让这些 self-attention 结构注意到补全的`[PAD]`部分，我们需要输入一个 attention_masks 标签，告诉模型哪些内容是真实内容，哪些是无意义的[PAD]。

刚刚说到被 `padding` 部分是不需要被 attention 到的。相当于这部分在  attention_masks 中的标签就是真实句子为1，padding部分为0。所以我们得到attention masks：

In [15]:
# 创建attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

In [16]:
# 第一句话的 attention_masks
attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

####  准备Labels

首先准备Labels。这些标题的 Labels 在一开始就已经分离开来，保存在了 `labels` 里面
 
这里可以用 `train_test_split` 来分。注意，由于多了一个 `attention_masks` 所以我们需要用两次 `train_test_split`，并且采用相同的随机种子。

In [17]:
print(len(labels))
print(labels[0:10])

180000
['3\n', '4\n', '1\n', '7\n', '5\n', '5\n', '9\n', '1\n', '8\n', '4\n']


由于现在的labels里面并不是数字，而且有换行符`\n`，我们需要一些处理：

In [18]:
clean_labels = []
for label in labels:
    clean_labels.append(int(label.strip('\n')))

print(clean_labels[0:10])

[3, 4, 1, 7, 5, 5, 9, 1, 8, 4]


In [19]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, clean_labels, 
                                                            random_state=2019, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2019, test_size=0.1)

In [20]:
train_labels[0:10]

[4, 3, 4, 3, 8, 8, 1, 8, 7, 7]

In [21]:
print(f"      标签总数：", len(labels))
print(f"训练集标签总数：", len(train_labels))
print(f"验证集标签总数：", len(validation_labels))

      标签总数： 180000
训练集标签总数： 162000
验证集标签总数： 18000


现在准备放入 PyTorch 中，准备 Tensor 化：

In [22]:
# tensor化
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [23]:
print(len(validation_inputs))
print(len(validation_labels))
print(len(validation_masks))

18000
18000
18000


#### 创建迭代器

我们采用

`torch.utils.data.TensorDataset` 将他们封装为 `TensorDataset` 的形式，

`torch.utils.data.RandomSampler` 采用随机采样的方法从中采样，

`torch.utils.data.DataLoader` 自动形成迭代器。

> 注意 batch_size 的设置大小和显存大小密切相关！

In [24]:
# batch size
batch_size = 16

In [25]:
# 形成训练数据集
train_data = TensorDataset(train_inputs, train_masks, train_labels)  
# 随机采样
train_sampler = RandomSampler(train_data) 
# 读取数据
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)


# 形成验证数据集
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
# 随机采样
validation_sampler = SequentialSampler(validation_data)
# 读取数据
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### BERT的微调

在准备好输入以后，现在我们开始微调BERT模型。

使用 `BertForSequenceClassification`，它就是一个普通BERT模型，在最后面加了一个线形层用于分类。

#### 导入模型

直接使用 `from_pretrained` 导入预训练好的中文 BERT 模型：

In [26]:
# 统计标签种类
label_count = len(set(labels))
print(label_count)

10


In [27]:
# 读取 BertForSequenceClassification 模型，
# 是一个预训练的BERT模型，在最后面加了一个线形层用于分类。

model = BertForSequenceClassification.from_pretrained("./bert-chinese/", 
                                                      num_labels=label_count)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### 准备微调

待补充。

其中，`no_decay`见[issue#492](https://github.com/huggingface/transformers/issues/492)

In [28]:
# BERT fine-tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']

# 权重衰减
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 
     'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 
     'weight_decay': 0.0}]

In [29]:
# 优化器
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=5e-5)

In [30]:
# 准确率计算函数
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [31]:
# 保存loss
train_loss_set = []
# epochs 
epochs = 4

#### 开始训练

4个epoch：

In [33]:
# BERT training loop
for _ in range(epochs): 
    ## 训练
    print(f"当前epoch： {_}")
    # 开启训练模式
    model.train()
    tr_loss = 0  # train loss
    nb_tr_examples, nb_tr_steps = 0, 0
    # Train the data for one epoch
    for step, batch in tqdm(enumerate(train_dataloader)):
        # 把batch放入GPU
        batch = tuple(t.to(device) for t in batch)
        # 解包batch
        b_input_ids, b_input_mask, b_labels = batch
        # 梯度归零
        optimizer.zero_grad()
        # 前向传播loss计算
        output = model(input_ids=b_input_ids, 
                       attention_mask=b_input_mask, 
                       labels=b_labels)  # 有labels的时候，且labels>1就直接返回Cross-Entropy
        loss = output[0]
        # print(loss)
        # 反向传播
        loss.backward()
        # Update parameters and take a step using the computed gradient
        # 更新模型参数
        optimizer.step()
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
    print(f"当前 epoch 的 Train loss: {tr_loss/nb_tr_steps}")

0it [00:00, ?it/s]

当前epoch： 0


10125it [22:26,  7.52it/s]
1it [00:00,  6.76it/s]

当前 epoch 的 Train loss: 0.31841557952945615
当前epoch： 1


10125it [22:32,  7.48it/s]
1it [00:00,  7.13it/s]

当前 epoch 的 Train loss: 0.2379983004839332
当前epoch： 2


10125it [22:20,  7.55it/s]
1it [00:00,  7.13it/s]

当前 epoch 的 Train loss: 0.22909607106667979
当前epoch： 3


10125it [22:20,  7.55it/s]

当前 epoch 的 Train loss: 0.19303164174012197


#### 验证数据集

In [34]:
# 验证状态
model.eval()

# 建立变量
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
# Evaluate data for one epoch

In [35]:
# 验证集的读取也要batch
for batch in tqdm(validation_dataloader):
    # 元组打包放进GPU
    batch = tuple(t.to(device) for t in batch)
    # 解开元组
    b_input_ids, b_input_mask, b_labels = batch
    # 预测
    with torch.no_grad():
        # segment embeddings，如果没有就是全0，表示单句
        # position embeddings，[0,句子长度-1]
        logits = model(input_ids=b_input_ids, 
                       attention_mask=b_input_mask,
                       token_type_ids=None,
                       position_ids=None)  
                       
    # print(logits[0])
    # Move logits and labels to CPU
    logits = logits[0].detach().cpu().numpy()  # 注意这里的logits是在softmax之前，所以和不为1
    label_ids = b_labels.to('cpu').numpy()
    # print(logits, label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)  # 计算准确率
    eval_accuracy += tmp_eval_accuracy  # 准确率积累
    nb_eval_steps += 1  # 步数积累
print(f"Validation Accuracy: {eval_accuracy/nb_eval_steps}")    

100%|██████████| 1125/1125 [00:34<00:00, 32.55it/s]

Validation Accuracy: 0.9240555555555555
